# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [43]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [44]:
# To load in data file "city data all.csv" from the first exercise WeatherPy.
file_to_load = "../WeatherPy/output_data/city_data_all.csv"
city_data_df = pd.read_csv(file_to_load)
city_data_df_all = pd.DataFrame(city_data_df)
city_data_df_all.head()

,City Name,City ID,Country,City Longitude,City Latitude,City Wind Speed,City Max Temperature,City Humidity,City Cloudiness,City Dates
0,Arlit,2447513,NE,7.3853,18.7369,12.26,89.26,4,0,1621982659
1,Hermanus,3366880,ZA,19.2345,-34.4187,7.31,57.90,74,51,1621982581
2,Hendek,745664,TR,30.7481,40.7994,1.95,50.32,81,3,1621982659
3,Dongsheng,1812256,CN,109.9776,39.8161,15.08,54.79,48,39,1621982659
4,Bathsheba,3374083,BB,-59.5167,13.2167,12.57,81.21,61,20,1621982595


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [45]:
gmaps.configure(api_key=g_key)
city_locations_all_location = city_data_df_all[["City Latitude", "City Longitude"]]
city_locations_all_humidity = city_data_df_all["City Humidity"].astype(float)

In [46]:
# Plot Heatmap
fig_1 = gmaps.figure(map_type="TERRAIN")
# Create heat layer
heat_layer = gmaps.heatmap_layer(city_locations_all_location, weights=city_locations_all_humidity, 
                                 dissipating=False, max_intensity=5,
                                 point_radius=1)
# Add layer
fig_1.add_layer(heat_layer)
# Display figure
fig_1

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [47]:
# To narrow down the "city_data" Dataframe to 70-80 degress, 0 cloudiness, and Wind Speed less than 10 mph.
future_vac_sites = (city_data_df_all.loc[city_data_df_all["City Max Temperature"] >= 70])
future_vac_sites_temp = (future_vac_sites.loc[future_vac_sites["City Max Temperature"] <= 80])
future_vac_sites_temp_cloudiness = (future_vac_sites_temp.loc[future_vac_sites_temp["City Cloudiness"] == 0])
future_vac_sites_temp_cloudiness_wind = (future_vac_sites_temp_cloudiness.loc[future_vac_sites_temp_cloudiness["City Wind Speed"] < 10])
future_vac_sites_temp_cloudiness_wind_humid = (future_vac_sites_temp_cloudiness_wind.loc[future_vac_sites_temp_cloudiness_wind["City Humidity"] < 80])
future_vac_sites_temp_cloudiness_wind_humid

,City Name,City ID,Country,City Longitude,City Latitude,City Wind Speed,City Max Temperature,City Humidity,City Cloudiness,City Dates
301,Al Bardīyah,80509,LY,25.0865,31.7561,3.15,72.91,48,0,1621982749
312,Kiryat Gat,293842,IL,34.7642,31.6100,2.62,78.40,55,0,1621982752
424,Birjand,140463,IR,59.2211,32.8663,6.91,70.27,14,0,1621982786
489,Sīwah,347863,EG,25.5195,29.2041,6.91,79.84,23,0,1621982804
496,Mossamedes,3347019,AO,12.1522,-15.1961,2.62,74.17,68,0,1621982806
530,Saint-Pierre,935214,RE,55.4781,-21.3393,8.05,73.04,77,0,1621982816


In [48]:
# Plotting the 6 cities which meet the crieria of 70-80 degress, 0 cloudiness, and Wind Speed less than 10 mph. 
gmaps.configure(api_key=g_key)
city_locations_ideal = future_vac_sites_temp_cloudiness_wind_humid[["City Latitude", "City Longitude"]]
city_humidity_ideal = future_vac_sites_temp_cloudiness_wind_humid["City Humidity"]

In [49]:
# Plotting the 6 cities which meet the crieria of 70-80 degress, 0 cloudiness, and Wind Speed less than 10 mph. 
fig_2 = gmaps.figure(map_type="TERRAIN")
# Create heat layer
heat_layer = gmaps.heatmap_layer(city_locations_ideal, weights=city_humidity_ideal, 
                                 dissipating=False, max_intensity=4,
                                 point_radius=1.5)
# Add Heat layer
fig_2.add_layer(heat_layer)
# Display figure
# fig_2

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [50]:
# To add a new blank column for "Hotel Name"
hotel_df = future_vac_sites_temp_cloudiness_wind_humid
hotel_df = hotel_df.copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City Name,City ID,Country,City Longitude,City Latitude,City Wind Speed,City Max Temperature,City Humidity,City Cloudiness,City Dates,Hotel Name
301,Al Bardīyah,80509,LY,25.0865,31.7561,3.15,72.91,48,0,1621982749,
312,Kiryat Gat,293842,IL,34.7642,31.6100,2.62,78.40,55,0,1621982752,
424,Birjand,140463,IR,59.2211,32.8663,6.91,70.27,14,0,1621982786,
489,Sīwah,347863,EG,25.5195,29.2041,6.91,79.84,23,0,1621982804,
496,Mossamedes,3347019,AO,12.1522,-15.1961,2.62,74.17,68,0,1621982806,
530,Saint-Pierre,935214,RE,55.4781,-21.3393,8.05,73.04,77,0,1621982816,


In [51]:
# Associaed Hotels for the 
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"radius": 5000, "types": "lodging", "key": g_key}

for index, row in hotel_df.iterrows():
    lat = row["City Latitude"]
    lng = row["City Longitude"]
    params["location"] = f"{lat}, {lng}"
    response = requests.get(base_url, params=params).json()
    print(json.dumps(response, indent=4, sort_keys=True))
    results = response["results"]
    try:
#     print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("NA")

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 31.7559067,
                    "lng": 25.0857804
                },
                "viewport": {
                    "northeast": {
                        "lat": 31.7574200802915,
                        "lng": 25.0870758302915
                    },
                    "southwest": {
                        "lat": 31.7547221197085,
                        "lng": 25.08437786970849
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "\u0628\u0648\u0631\u062a\u0648 \u0628\u0631\u062f\u064a\u0629",
            "photos": [
                {
                    "height": 960,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/cont

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "CLOSED_TEMPORARILY",
            "geometry": {
                "location": {
                    "lat": 31.613093,
                    "lng": 34.716021
                },
                "viewport": {
                    "northeast": {
                        "lat": 31.6144419802915,
                        "lng": 34.7173699802915
                    },
                    "southwest": {
                        "lat": 31.6117440197085,
                        "lng": 34.7146720197085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "OlusHome",
            "permanently_closed": true,
            "place_id": "ChIJQd43S_aQAhURVqnCVKhocNs",
            "plus_code": {
                "compound_code": "JP78+6C Nir Hen, Israel",
                "global_code": "8G3PJP78+6C"
          

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIBmFb000Wt8lPOQJaUnkmx4HOGWtS5mzlrXGpsrL-r7ncPmwaiFgww6WX8vjUmj-3ET8ngfEFcGGEGNs6a413ShF84i8xlzj0VW6QYpSpqQxk6jLUEq-P2ekEkIM2hjuayR8pKJKiW5eRxZhtSlqNv8SENkmwF25b6eoBvrV8LwSpABgENAoIUb319z07H3ER11waLSDeS75wuPGWjqWu6tSHomt_mX64TMVeeRtFjf0vJkPvGrKQAnA3gMMbsph6VVoZ95oCNjW65Slu8YgOX1WST8xYjs7YET9B8YHxFhB2M25Ko23m9sJn5ypMxvNMjIRdJfqLc9JFWLHXhxllz-VmLYjxZ_YEMUNoN1jigx7Q3fr7jQcIj5qKEjqZsRbCwlzbAgm2rOLIWLcJk4dbXiYkm-iEMsrMZOdrpd1PsvwGwIL-6bnjJ",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 32.8713752,
                    "lng": 59.2307798
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.8728181302915,
                        "lng": 59.2321445802915
                    },
                    "southwest": {
                        "lat": 32.8701201697085,
    

{
    "html_attributions": [],
    "next_page_token": "ATtYBwLNEgGK4tBtQTKb8sTVGp4oZAFn4HyPWtr-47W515kJhMcTHA7dXWvVa87nr0BDoj2pQxpommpRJT7m5Uj1ohTNMSn60b-nDdaHqSCY4_uVj0OKLP2pNc-6-G4vVUMN5B0tlMj3k3PWg_14vAQhstvTCMWj0rVSFOBzzWAJb6ycFCkdLBbbZn-MWIxbKTbfdqL6PsxBkbpfzM1vXfSWcbZIH520oa-4kl9slyb5CfYZlZ1Dgz0OV5YfZ309PJXycUKZCecuBxsL7kz7qZXHPdNfbzeYzAsnZjmTFAmePq-2bX5oFecyCSnkBl3o9wI80Y1lQn9Bff5tgolHpvD8tQ2VhNVDnvwkUXZj3xjpDneDTGZZcDw7JreANL2mnYUS1H-rFKWVzn-qcJaGAmts88AyPJ6qXSINT1x3mbCmRQGU3Kmba3iaV7DO",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 29.1885457,
                    "lng": 25.5433359
                },
                "viewport": {
                    "northeast": {
                        "lat": 29.1898560802915,
                        "lng": 25.5447542802915
                    },
                    "southwest": {
                        "lat": 29.1871581197085,
    

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIuABHZIZZ4eEDoVSckcvR1J9oekBt50BFu2upKnI1tUSZtte6Vz_SvuIWUaLvkhZb8xqY7zjK7Hgcg6tQG_bBpdRj4-X7tazkwurzCxpxU9_xVGsKcb-v2rod7Ebk1aatgz1o5ucu7UVqhygHFHthsTFmTxg4lSiqjC5SAFlDupwWHHk5JcYcUKbmLuUJML7r3Mg-a5sPhIp93gRZMt6l7n4Vp1FwlVT2HO8GEMadb66_sIC_fZ_enYKBNMNCZ07MvC3t0Cu9Mxv1iUvAODCWOENcKkfUERrnxXXdurSLQZEvkizwpksB_Uol1wF3-MqhM2T9DSOY70yzQjfjNsYz7ebXI0-Jspzn3OMF5-YGEsGfhvSgjnPcuanOb7Dbjdmf5yZ1vxX5uj3FiLk_du7WSvuR2Gekk0_KX2yEze7c4A7CjpQvgtbjF",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -15.1978317,
                    "lng": 12.1575544
                },
                "viewport": {
                    "northeast": {
                        "lat": -15.1965399197085,
                        "lng": 12.1588349802915
                    },
                    "southwest": {
                        "lat": -15.1992378802915,
 

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIKCD4nac3jbPXxQSpDQMAnV_to8hsKTEUot0T3BNVNDd3NLL-S77efrUK61uuKnpCQnMnleh-O1KeZ72uwISGCn9lAv9037MnQLCLQoAjgxmr8z1kJHA2qy85sFfOeWROYLHU8GswQnCZ8oPPL4hc_whSH4IbQdJruxZ_e88w9J5kU9xjwQ5nU7yyVuqwlfvDerUQe_VIfVPYAy8elp2-WC4mB4D1pBXPKqNxpwVc9EP3AbaH4AXpCob1znNw7rvnFmxQ8c0eH6EDXDdFNp08S7m5R3HO56ZU98dfXlb_r9vxQb0kOU-6ZB87RiXFalO0JFvvBIoqQPXU7F5zG9NmXUGKVRQ3dQKrXjvjt6FXNtqhZy2Zsb5VCcgzTlMWrp6zpunCi1yTckpnFpTKkH8F2r-4T_2FtD6Xy94mMg27ezU4Ouz0lp9Y6",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -21.3403053,
                    "lng": 55.4722328
                },
                "viewport": {
                    "northeast": {
                        "lat": -21.3389563197085,
                        "lng": 55.4735817802915
                    },
                    "southwest": {
                        "lat": -21.34165428029151,


In [52]:
# Adding hotel name to the DataFrame of 6 cities which meet the crieria of 70-80 degress, 0 cloudiness, and Wind Speed less than 10 mph.
hotel_df

,City Name,City ID,Country,City Longitude,City Latitude,City Wind Speed,City Max Temperature,City Humidity,City Cloudiness,City Dates,Hotel Name
301,Al Bardīyah,80509,LY,25.0865,31.7561,3.15,72.91,48,0,1621982749,بورتو بردية
312,Kiryat Gat,293842,IL,34.7642,31.6100,2.62,78.40,55,0,1621982752,OlusHome
424,Birjand,140463,IR,59.2211,32.8663,6.91,70.27,14,0,1621982786,Birjand Tourist Hotel
489,Sīwah,347863,EG,25.5195,29.2041,6.91,79.84,23,0,1621982804,Qasr Alzaytuna Inn
496,Mossamedes,3347019,AO,12.1522,-15.1961,2.62,74.17,68,0,1621982806,Hotel Chik Chik Namibe
530,Saint-Pierre,935214,RE,55.4781,-21.3393,8.05,73.04,77,0,1621982816,Lindsey Hôtel


In [53]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["City Latitude", "City Longitude"]]

In [42]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig_2.add_layer(markers)

# Display figure
fig_2

Figure(layout=FigureLayout(height='420px'))